# Identificador de Arquivos Relevantes

Treinamento de um modelo de aprendizado de máquina supervisionado para identificar, em um conjunto de arquivos PDF, quais desses arquivos são relevantes em um contexto específico.

Utiliza como dados de treinamento e teste uma base de documentos pré-classificados.

**Técnicas aplicadas:**
* Processamento:
> * Remoção de dados nulos
> * Balanceamento de classes
* Particionamento de dados treino e teste (+validação com cross_validation)
* Pipeline
* Modelos de classificação
* Matrix de confusão

## Sumário
1. [Importações e configurações](#p1)
2. [Exploração dos dados](#p2)
3. [Processamento](#p3)
4. [Treinamento e comparativo de modelos](#p4)
6. [Exportação](#p5)

<a id="p1"></a>
## 1. Importações e configurações

In [ ]:
import pandas as pd

# Modelos de classificação
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.naive_bayes import MultinomialNB

# Processamento
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

# Avalição 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from matplotlib import pyplot as plt
#fig, ax = plt.subplots(figsize=(20, 20))

In [ ]:
# Remove warnings de tratamento de slices de data frames
# https://stackoverflow.com/questions/37841525/correct-way-to-set-value-on-a-slice-in-pandas
pd.set_option('mode.chained_assignment', None)

In [ ]:
# Dados de entrada

#data_file = '/home/ubuntu/nlp/notebooks/test.csv'
data_file = '/home/ubuntu/nlp/notebooks/pdf-status.csv'

df = pd.read_csv(data_file, sep=',', low_memory=False, 
                 names=['PDF_File_Path', 'Relevant', 'Content']
                     #, nrows=10)
                    )

df.head()

<a id="p2"></a>
# 2. Exploração dos dados

In [ ]:
type(df)

In [ ]:
# Tamanho do dataset
df.shape

In [ ]:
df.head()

In [ ]:
# Tipos das colunas
df.dtypes

In [ ]:
# Contador
df['Relevant'].value_counts()

In [ ]:
# Percentual
df['Relevant'].value_counts(normalize=True)

In [ ]:
# Valores nulos
100 * df.isna().sum() / len(df)

<a id="p3"></a>
# 3. Processamento

### Avaliação de conteúdo nulo

In [ ]:
# Remove nulos
dfProc = df.dropna(subset=['Relevant', 'Content'])
100 * dfProc.isna().sum() / len(dfProc)

In [ ]:
# Percentual da amostra limpa relacionada a total
print(f"Amonstra total (100.00%): {df.Content.count()}")
print(f"Amonstra limpa ({100 * dfProc.Content.count()/df.Content.count():.4}%): {dfProc.Content.count()}")

### Balenceamento de classes

Observou-se que a amostra apresenta classes desbalanceadas, com uma quantidade de amostras de valor 'Relevant' 1 muito menor que a quantidade de amostras com valor 0. Dessa forma, será realizado um ressample da amostra (down sample da amostra majoritária).

In [ ]:
# Classe majoritária
dfProcMaj = dfProc[dfProc['Relevant'] == 0]
majCount = len(dfProcMaj)
majCount

In [ ]:
# Classe minoritária
dfProcMin = dfProc[dfProc['Relevant'] == 1]
minCount = len(dfProcMin)
minCount

In [ ]:
# Down sample
# Classe minoritária como 1/3 da classe majoritária
dfProcMaj_DownSampled = dfProcMaj.sample(n=minCount*3, replace=True, random_state=42)

dfProcResampled = pd.concat([dfProcMin, dfProcMaj_DownSampled])

In [ ]:
dfProcResampled.head()

In [ ]:
dfProcResampled['Relevant'].value_counts()

In [ ]:
dfProcResampled['Relevant'].value_counts(normalize=True)

In [ ]:
# Percentual da amostra balenceada VS da amostra total
len(dfProcResampled) / len(df)

<a id="p4"></a>
# 4. Treinamento e comparativo de modelos

### Testes com modelos de classificação

In [ ]:
# Separa entrada de saída (X, y)
X = dfProcResampled['Content']
y = dfProcResampled['Relevant']

In [ ]:
# Separa dados de treino e de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Comparativo de modelos

O comparativo entre os modelos será reliazado via cross_val_score(). Essa técnica (https://scikit-learn.org/stable/modules/cross_validation.html) divide a amostra em três partes, permitindo que o treinamento, avaliação inicial e avaliação final sejam realizados com sets diferentes.

**Naive Bayes**

In [ ]:
# Naïve Bayes
nb_pipeline = Pipeline([
                        ('tfidf', TfidfVectorizer()), 
                        ('clf', MultinomialNB()),
                       ])

In [ ]:
nb_pipeline.fit(X_train, y_train)

In [ ]:
# Predictions set
predictions = nb_pipeline.predict(X_test)

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, predictions))

In [ ]:
plot_confusion_matrix(nb_pipeline, X_test, y_test);
plt.show()

In [ ]:
# Classification report
print(metrics.classification_report(y_test, predictions))

In [ ]:
# Overall accuracy
print(metrics.accuracy_score(y_test, predictions))

In [ ]:
cross_val_score(nb_pipeline, X, y, cv=5, scoring='accuracy').mean()

**Linear SVC**

In [ ]:
# Linear SVC
lsvc_pipeline = Pipeline([
                            ('tfidf', TfidfVectorizer()),
                            ('clf', LinearSVC()),
                        ])

In [ ]:
lsvc_pipeline.fit(X_train, y_train)

In [ ]:
# Prediction set
predictions = lsvc_pipeline.predict(X_test)

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, predictions))

In [ ]:
plot_confusion_matrix(lsvc_pipeline, X_test, y_test);
plt.show()

In [ ]:
# Classification report
print(metrics.classification_report(y_test, predictions))

In [ ]:
# Overall accuracy
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
cross_val_score(lsvc_pipeline, X, y, cv=5, scoring='accuracy').mean()

**DecisionTreeClassifier**

In [ ]:
# DTC
dtc_pipeline = Pipeline([
                            ('tfidf', TfidfVectorizer()),
                            ('clf', DecisionTreeClassifier(max_depth=30)),
                        ])

In [ ]:
dtc_pipeline.fit(X_train, y_train)

In [ ]:
# Prediction set
predictions = dtc_pipeline.predict(X_test)

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(y_test, predictions))

In [ ]:
plot_confusion_matrix(lsvc_pipeline, X_test, y_test);
plt.show()

In [ ]:
plot_confusion_matrix(dtc_pipeline, X_test, y_test);

In [ ]:
metrics.f1_score(y_test, dtc_pipeline.predict(X_test), average='weighted')

In [ ]:
# Classification report
print(metrics.classification_report(y_test, predictions))

In [ ]:
# Overall accuracy
print(metrics.accuracy_score(y_test,predictions))

In [ ]:
cross_val_score(dtc_pipeline, X, y, cv=5, scoring='accuracy').mean()

<a id="p5"></a>
# 5. Exportação

In [ ]:
# Seleção do modelo mais eficaz
model = dtc_pipeline

In [ ]:
import joblib

# Save to file in the current working directory
joblib_file = "check_relevance_mlmodel.pkl"
joblib.dump(model, joblib_file)

In [ ]:
# Exemplo de utilização do modelo
# Load from file
joblib_model = joblib.load(joblib_file)

# Calculate the accuracy and predictions
score = joblib_model.score(X_test, y_test)
print("Test score: {0:.2f} %".format(100 * score))

y_predict = joblib_model.predict(X_test)
y_predict